#

In [1]:
import pandas as pd
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.tag.mapping import map_tag
nltk.download('universal_tagset')
from nltk import pos_tag
nltk.download('averaged_perceptron_tagger')
nltk.download('sentiwordnet')
from nltk.corpus import sentiwordnet as swn
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\anush\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\anush\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\anush\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\anush\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package sentiwordnet to
[nltk_data]     C:\Users\anush\AppData\Roaming\nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!


In [2]:
tweets = pd.read_csv("Datasets/processed_tweets.csv")
tweets = tweets.drop(['Unnamed: 0'], axis= 1)
tweets

,Text,Date,Year,hashtag,Cleaned
0,No violation by @narendramodi :Election Commis...,2014-10-11,2014,['MannKiBaat'],violation election commission
1,#PMModi's #MannKiBaat radio show: #ElectionCom...,2014-10-10,2014,"['PMModi', 'MannKiBaat', 'ElectionCommission']",radio show says broadcast violate model code c...
2,#MannKiBaat #Sobtian #FanGal\nDil Humko Kabhi ...,2014-10-10,2014,"['MannKiBaat', 'Sobtian', 'FanGal', 'BarunSobti']",humko kabhi samjhata haihum dilko kabhi samjhate
3,#Congress lodges complaint with #EC against #M...,2014-10-06,2014,"['Congress', 'EC', 'Modi', 'MannKiBaat']",lodges complaint with against program
4,#Congress lodges complaint with EC against PM ...,2014-10-06,2014,"['Congress', 'NarendraModi', 'MannKiBaat']",lodges complaint with against program india radio
...,...,...,...,...,...
10326,#MannKiBaat\nWorld’s largest vaccination progr...,2021-03-30,2021,['MannKiBaat'],largest vaccination programmevia namo
10327,#MannKiBaat\nSacrifices that inspire us toward...,2021-03-30,2021,['MannKiBaat'],that inspire towards dutiesvia namo
10328,#MannKiBaat\nNari Shakti leading from the fron...,2021-03-30,2021,['MannKiBaat'],shakti leading from frontvia namo
10329,#MannKiBaat\n75th episode at the start of 75 y...,2021-03-30,2021,['MannKiBaat'],episode start years independencevia namo


In [3]:
pstem = PorterStemmer()
lem = WordNetLemmatizer()
label=list()
pos=[]
neg=[]
neu=[]
for i in range(len(tweets.index)):
  positive_score=0
  negative_score=0
  
  
  text = tweets.loc[i]['Cleaned']
  tokens = nltk.word_tokenize(text)
  tagged_sent = pos_tag(tokens)
  store_it = [(word, map_tag('en-ptb', 'universal', tag)) for word, tag in tagged_sent]
  for word,tag in store_it:
    if(tag=='NOUN'):
        tag='n'
    elif(tag=='VERB'):
        tag='v'
    elif(tag=='ADJ'):
        tag='a'
    elif(tag=='ADV'):
        tag = 'r'
    else:
        tag='nothing'

    if(tag!='nothing'):
      concat=word+'.'+tag+'.01'
      #print(swn.senti_synset(concat))
      try:
        check=swn.senti_synset(concat)
        positive_score+=check.pos_score()
        negative_score+=check.neg_score()
      
      except Exception as e:
        wor = lem.lemmatize(word)
        concat = wor+'.'+tag+'.01'
        # Checking if there's a possiblity of lemmatized word be accepted into SWN corpus
        try:
            positive_score+=swn.senti_synset(concat).pos_score()
            negative_score+=swn.senti_synset(concat).neg_score()
            
        except Exception as e:
            wor = pstem.stem(word)
            concat = wor+'.'+tag+'.01'
            # Checking if there's a possiblity of lemmatized word be accepted
            try:
                positive_score+=swn.senti_synset(concat).pos_score()
                negative_score+=swn.senti_synset(concat).neg_score()
                
            except:
                
                continue
    #check=swn.senti_synset(concat)
    #if check.PosScore>check.NegScore:
      #positive_score+=1
    #elif check.NegScore>check.PosScore
  if positive_score>negative_score:
    label.append(1)
  elif negative_score>positive_score:
    label.append(-1)
  else:
    label.append(0)
  pos.append(positive_score)
  neg.append(negative_score)
  
  #print(label)
tweets['label']=label
tweets['positive']=pos
tweets['negative']=neg  
   
tweets

,Text,Date,Year,hashtag,Cleaned,label,positive,negative
0,No violation by @narendramodi :Election Commis...,2014-10-11,2014,['MannKiBaat'],violation election commission,0,0.000,0.00
1,#PMModi's #MannKiBaat radio show: #ElectionCom...,2014-10-10,2014,"['PMModi', 'MannKiBaat', 'ElectionCommission']",radio show says broadcast violate model code c...,1,0.250,0.00
2,#MannKiBaat #Sobtian #FanGal\nDil Humko Kabhi ...,2014-10-10,2014,"['MannKiBaat', 'Sobtian', 'FanGal', 'BarunSobti']",humko kabhi samjhata haihum dilko kabhi samjhate,0,0.000,0.00
3,#Congress lodges complaint with #EC against #M...,2014-10-06,2014,"['Congress', 'EC', 'Modi', 'MannKiBaat']",lodges complaint with against program,0,0.000,0.00
4,#Congress lodges complaint with EC against PM ...,2014-10-06,2014,"['Congress', 'NarendraModi', 'MannKiBaat']",lodges complaint with against program india radio,0,0.000,0.00
...,...,...,...,...,...,...,...,...
10326,#MannKiBaat\nWorld’s largest vaccination progr...,2021-03-30,2021,['MannKiBaat'],largest vaccination programmevia namo,1,0.375,0.25
10327,#MannKiBaat\nSacrifices that inspire us toward...,2021-03-30,2021,['MannKiBaat'],that inspire towards dutiesvia namo,0,0.000,0.00
10328,#MannKiBaat\nNari Shakti leading from the fron...,2021-03-30,2021,['MannKiBaat'],shakti leading from frontvia namo,0,0.000,0.00
10329,#MannKiBaat\n75th episode at the start of 75 y...,2021-03-30,2021,['MannKiBaat'],episode start years independencevia namo,0,0.000,0.00


In [4]:
tweets['label'].value_counts()

 0    4475
 1    4114
-1    1742
Name: label, dtype: int64

In [188]:
months = []
for i in range(len(tweets)):
    months.append(tweets["Date"][i][2:7])    
tweets['Month'] = months

month_group = tweets.groupby('Month')['label'].value_counts(sort=False)
print(month_group)

year_group = tweets.groupby('Year')['label'].value_counts(sort=False)
print(year_group)

Month  label
14-10  -1        15
        0        43
        1        51
14-11  -1        62
        0       181
               ... 
21-02   0        86
        1        36
21-03  -1        28
        0        70
        1        85
Name: label, Length: 204, dtype: int64
Year  label
2014  -1       158
       0       361
       1       387
2015  -1       202
       0       668
       1       528
2016  -1       300
       0       676
       1       665
2017  -1       217
       0       634
       1       416
2018  -1       181
       0       681
       1       657
2019  -1       229
       0       565
       1       554
2020  -1       376
       0       677
       1       704
2021  -1        79
       0       213
       1       203
Name: label, dtype: int64


In [205]:
cols = ['Year', 'Negative', 'Neutral', 'Positive']
calculated = pd.DataFrame(columns=cols)
year = []
# month = []
negative = []
neutral = []
positive = []

for i in range(0, len(year_group), 3) :
    year.append(year_group.index[i][0])
#     month.append(month_group.index[i][0][0:])
    negative.append(year_group.iloc[i])
    neutral.append(year_group.iloc[i+1])
    positive.append(year_group.iloc[i+2])
    
calculated['Year'] = year
# calculated['Month'] = month
calculated['Negative'] = negative
calculated['Neutral'] = neutral
calculated['Positive'] = positive

In [206]:
calculated['Sum'] = calculated.iloc[:, 1:].sum(axis=1)
calculated['% Negative'] = calculated['Negative']/calculated['Sum']*100
calculated['% Neutral'] = calculated['Neutral']/calculated['Sum']*100
calculated['% Positive'] = calculated['Positive']/calculated['Sum']*100

In [207]:
calculated

,Year,Negative,Neutral,Positive,Sum,% Negative,% Neutral,% Positive
0,2014,158,361,387,906,17.439294,39.845475,42.715232
1,2015,202,668,528,1398,14.449213,47.782546,37.768240
2,2016,300,676,665,1641,18.281536,41.194394,40.524071
3,2017,217,634,416,1267,17.127072,50.039463,32.833465
4,2018,181,681,657,1519,11.915734,44.832126,43.252140
5,2019,229,565,554,1348,16.988131,41.913947,41.097923
6,2020,376,677,704,1757,21.400114,38.531588,40.068298
7,2021,79,213,203,495,15.959596,43.030303,41.010101


In [214]:
import plotly
import plotly.graph_objects as go
x = list(calculated['Year'])
fig = go.Figure()
fig.add_bar(x=x, y=list(calculated['% Negative']), name='Negative reviews', marker=dict(
        color='rgba(255,0,0, 0.7)'))
fig.add_bar(x=x, y=list(calculated['% Neutral']), name='Neutral reviews', marker=dict(
        color='rgba(0,0,255, 0.7)'))
fig.add_bar(x=x, y=list(calculated['% Positive']), name='Positive reviews', marker=dict(
        color='rgba(0,255,0, 0.7)'))
fig.update_layout(barmode="relative", yaxis=dict(
        title_text="Reviews %",
        ticktext=["0%", "20%", "40%", "60%","80%","100%"],
        tickvals=[0, 20, 40, 60, 80, 100],
    ))
fig.show()